# Mise en place d'un algo pour trouver les croisements

In [2]:
!pip install osmnx --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 2.7 MB/s eta 0:00:00


In [3]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import unicodedata



In [9]:
# Télécharger tout le graphe routier de Paris
G = ox.graph_from_place("Paris, France", network_type='drive')
gdf_rues = ox.graph_to_gdfs(G, nodes=False, edges=True)
gdf_rues = gdf_rues[gdf_rues["name"].notna()]  # Garder uniquement les rues nommées


KeyboardInterrupt: 

In [5]:
def normalize_street_name(name):
    if isinstance(name, list):
        name = name[0] if name else ''
    elif name is None:
        name = ''
    return unicodedata.normalize('NFKD', str(name)).encode('ASCII', 'ignore').decode('utf-8').upper()

gdf_rues['name'] = gdf_rues['name'].apply(normalize_street_name)


In [10]:
# Tes coordonnées de test (ajoute autant que tu veux)
coordinates = [
    (48.8480778, 2.3505051),
    (48.8523487, 2.3482382),
    (48.8560699, 2.3507047),
]

gdf_points = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon in coordinates], crs="EPSG:4326")

buffer_radius = 0.0002  # ~20m
results = []

for idx, point in gdf_points.iterrows():
    buffer = point.geometry.buffer(buffer_radius)
    matches = gdf_rues[gdf_rues.intersects(buffer)]
    matched_names = matches["name"].unique().tolist()
    results.append({
        "point": coordinates[idx],
        "matched_rues": matched_names
    })

for r in results:
    print(f"📍 Point {r['point']} → {', '.join(r['matched_rues']) if r['matched_rues'] else '❌ Aucune rue détectée'}")


📍 Point (48.8480778, 2.3505051) → RUE MONGE, RUE DES ECOLES
📍 Point (48.8523487, 2.3482382) → QUAI DE MONTEBELLO, RUE LAGRANGE
📍 Point (48.8560699, 2.3507047) → QUAI DE L'HOTEL DE VILLE, PLACE DE L'HOTEL DE VILLE, PONT D'ARCOLE, QUAI DE GESVRES


In [7]:
# Tes coordonnées de test (ajoute autant que tu veux)
coordinates = [
    (48.8748138, 2.3573832),
    (48.8689741, 2.3557506),
    (48.8696795, 2.3507047),
]

gdf_points = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon in coordinates], crs="EPSG:4326")

buffer_radius = 0.0002  # ~20m
results = []

for idx, point in gdf_points.iterrows():
    buffer = point.geometry.buffer(buffer_radius)
    matches = gdf_rues[gdf_rues.intersects(buffer)]
    matched_names = matches["name"].unique().tolist()
    results.append({
        "point": coordinates[idx],
        "matched_rues": matched_names
    })

for r in results:
    print(f"📍 Point {r['point']} → {', '.join(r['matched_rues']) if r['matched_rues'] else '❌ Aucune rue détectée'}")


📍 Point (48.8748138, 2.3573832) → BOULEVARD DE MAGENTA, BOULEVARD DE STRASBOURG, RUE DE LA FIDELITE
📍 Point (48.8689741, 2.3557506) → BOULEVARD SAINT-DENIS, BOULEVARD SAINT-MARTIN, RUE SAINT-MARTIN, CARREFOUR DES THEATRES
📍 Point (48.8696795, 2.3507047) → RUE BEAUREGARD, RUE DE LA LUNE


In [11]:
# Tes coordonnées de test (ajoute autant que tu veux)
coordinates = [
    (48.8804947, 2.3523043),
    (48.8719982, 2.3400527),
    (48.8722422, 2.3382998),
    (48.8729396,2.3332569),
    (48.8742709,2.3313146),
    (48.8742988,2.3327119),
    (48.8761423,2.3311152)
]

gdf_points = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon in coordinates], crs="EPSG:4326")

buffer_radius = 0.0002  # ~20m
results = []

for idx, point in gdf_points.iterrows():
    buffer = point.geometry.buffer(buffer_radius)
    matches = gdf_rues[gdf_rues.intersects(buffer)]
    matched_names = matches["name"].unique().tolist()
    results.append({
        "point": coordinates[idx],
        "matched_rues": matched_names
    })

for r in results:
    print(f"📍 Point {r['point']} → {', '.join(r['matched_rues']) if r['matched_rues'] else '❌ Aucune rue détectée'}")


📍 Point (48.8804947, 2.3523043) → BOULEVARD DE MAGENTA, RUE SAINT-VINCENT DE PAUL, RUE DE MAUBEUGE, RUE DE DUNKERQUE
📍 Point (48.8719982, 2.3400527) → RUE DE RICHELIEU, BOULEVARD HAUSSMANN, RUE DROUOT, BOULEVARD MONTMARTRE, BOULEVARD DES ITALIENS
📍 Point (48.8722422, 2.3382998) → BOULEVARD HAUSSMANN, RUE LE PELETIER
📍 Point (48.8729396, 2.3332569) → RUE DE LA CHAUSSEE-D'ANTIN, BOULEVARD HAUSSMANN, PLACE THEOPHILE BADER, RUE HALEVY, RUE LA FAYETTE
📍 Point (48.8742709, 2.3313146) → RUE DE PROVENCE, RUE DE MOGADOR
📍 Point (48.8742988, 2.3327119) → RUE DE PROVENCE, RUE DE LA CHAUSSEE-D'ANTIN
📍 Point (48.8761423, 2.3311152) → RUE SAINT-LAZARE, RUE DE MOGADOR


# Création du fichier pour les croisement


In [13]:
# 📦 Si nécessaire, installe les dépendances :
# !pip install geopandas openpyxl pandas shapely

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point


In [15]:
from google.colab import files
uploaded = files.upload()


Saving intersections.geojson to intersections.geojson


In [16]:
# 📦 Assurez-vous que ces libs sont installées si ce n'est pas déjà fait :
# !pip install osmnx geopandas

import osmnx as ox
import geopandas as gpd

# 📂 1. Charger tes points d’intersection depuis le fichier que TU uploades dans Colab
gdf_points = gpd.read_file("intersections.geojson")

# 🌍 2. Télécharger le graphe routier complet de Paris
G = ox.graph_from_place("Paris, France", network_type='drive')

# 🛣️ 3. Extraire les rues avec leurs géométries
gdf_rues = ox.graph_to_gdfs(G, nodes=False, edges=True)
gdf_rues = gdf_rues[gdf_rues["name"].notna()]  # Garder uniquement les rues nommées

# 🧭 4. S’assurer que tout est en EPSG:4326
gdf_points = gdf_points.to_crs(epsg=4326)
gdf_rues = gdf_rues.to_crs(epsg=4326)


In [18]:
!pip install unidecode --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.4 MB/s eta 0:00:00


In [20]:
from unidecode import unidecode  # pour enlever les accents

buffer_radius = 0.0002  # ~20 mètres
results = []

def nettoyer_nom(nom):
    """Nettoyage du nom de rue : supprime accents, type de voie, espaces"""
    if isinstance(nom, list):
        noms_plats = [nettoyer_nom(x) for x in nom]
        return noms_plats
    nom = unidecode(nom.upper())
    for mot in ["RUE ", "AVENUE ", "AV ", "BD ", "BOULEVARD ", "PLACE ", "CHEMIN ", "IMPASSE "]:
        nom = nom.replace(mot, "")
    return nom.strip().replace(" ", "_")

for idx, point in gdf_points.iterrows():
    buffer = point.geometry.buffer(buffer_radius)
    matches = gdf_rues[gdf_rues.intersects(buffer)]

    noms_raw = matches["name"].dropna().tolist()

    noms_nets = []
    for entry in noms_raw:
        if isinstance(entry, list):
            noms_nets.extend(entry)  # ajoute chaque sous-nom de la liste
        else:
            noms_nets.append(entry)

    noms_nets = [nettoyer_nom(n) for n in noms_nets]
    noms_nets = sorted(set(n for nom in noms_nets for n in (nom if isinstance(nom, list) else [nom])))[:2]  # max 2 noms

    if len(noms_nets) == 2:
        croisement = f"CROIST_{noms_nets[0]}_{noms_nets[1]}"
    elif len(noms_nets) == 1:
        croisement = f"CROIST_{noms_nets[0]}"
    else:
        croisement = "INCONNU"

    results.append({
        "id": tuple(round(coord, 7) for coord in (point.geometry.y, point.geometry.x)),
        "nom": croisement
    })


In [21]:
# ✅ Convertir en DataFrame et exporter
df_result = pd.DataFrame(results)
df_result.to_excel("liste_croisement.xlsx", index=False)

print("✅ Export terminé → liste_croisement.xlsx")


✅ Export terminé → liste_croisement.xlsx
